base_model 허깅페이스(https://huggingface.co/Anwarkh1/Skin_Cancer-Image_Classification)
모델 가중치 업데이트 불가능 상태로 테스트
데이터 : 이마(색소침착, 0-5)
가중치 : 데이터 분포에 따른 가중치 적용


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms, models
from transformers import AutoModelForImageClassification

In [2]:
from transformers import AutoModel, AutoTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def Image_feature_selector(data_sets, enhance):
    import cv2
    import numpy as np
    from PIL import Image  # PIL 추가

    new_image = {}
    file_name = data_sets["Images"].keys()
    for file_key in file_name:

        # 이미지 불러오기
        image = data_sets['Images'][file_key]

        # 어두운 영역을 감지하기 위해 이미지를 그레이스케일로 변환
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 이미지의 평균 밝기 계산
        mean_brightness = np.mean(gray_image) + enhance

        # 평균 밝기를 기준으로 어두운 영역 마스크 생성
        _, dark_mask = cv2.threshold(gray_image, mean_brightness - 20, 255, cv2.THRESH_BINARY_INV)

        # 어두운 영역을 강조하기 위해 밝기와 대비 증가
        enhanced_image = image.copy()
        enhanced_image[dark_mask > 0] = cv2.addWeighted(enhanced_image[dark_mask > 0], 1.8, np.zeros_like(enhanced_image[dark_mask > 0]), 0, 60)

        # numpy 배열을 PIL.Image로 변환
        new_image[file_key] = Image.fromarray(cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))  # OpenCV는 BGR, PIL은 RGB 사용

    return new_image

In [5]:
# GPU 여부 확인
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [6]:
# 데이터 준비 1. 핸드폰, 전면, 이마, 메타데이터 전체
# 테스트데이터
path = "/content/drive/MyDrive/Human_Final_Project/Data/Train_Data_Sets_02.pkl"
with open(path, "rb") as pickle_file:
    train_data_sets = pickle.load(pickle_file)
    print("데이터 로드 완료!!")

데이터 로드 완료!!


In [7]:
# 검증데이터
path = "/content/drive/MyDrive/Human_Final_Project/Data/Val_Data_Sets_02.pkl"
with open(path, "rb") as pickle_file:
    val_data_sets = pickle.load(pickle_file)
    print("데이터 로드 완료!!")

데이터 로드 완료!!


In [8]:
# 이미지 이름 추출
file_name = train_data_sets['Images'].keys()
file_list = []
for file_keys in file_name:
    file_list.append(file_keys)

In [9]:
#이미지 강조
new_image = Image_feature_selector(train_data_sets, 10)
new_val_image = Image_feature_selector(val_data_sets, 10)

In [10]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 모델 입력 크기에 맞추기
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 기준 정규화
])

In [11]:
class CustomDataset(Dataset):
    def __init__(self, data_sets, new_image, transform=transform):
        self.images = new_image
        self.metadata = data_sets['Metadata']
        self.keys = list(self.images.keys())
        self.transform = transform

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        image = self.images[key]
        if self.transform:
            image = self.transform(image)

        target = torch.tensor(self.metadata[key]['annotations']['forehead_pigmentation'], dtype=torch.float32)
        return image, target

In [12]:
#데이터셋 생성 (텐서형태로 변환)
dataset = CustomDataset(train_data_sets, new_image)
val_dataset = CustomDataset(val_data_sets, new_val_image)

In [28]:
#로더를 이용해 데이터 배치화
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

In [33]:
class SkinNet(nn.Module):
    def __init__(self):
        super(SkinNet, self).__init__()
        # 사전 학습된 모델 로드
        self.base_model = AutoModelForImageClassification.from_pretrained(
            "Anwarkh1/Skin_Cancer-Image_Classification",
            ignore_mismatched_sizes=True
        )

        # 사전 학습된 모델의 파라미터 고정
        for param in self.base_model.parameters():
            param.requires_grad = False

        # 사전 학습된 모델의 출력 크기 확인
        num_features = self.base_model.classifier.out_features

        # 사용자 정의 레이어 추가
        self.fc1 = nn.Linear(num_features, 1000)  # 128은 임의로 설정한 크기
        self.fc2 = nn.Linear(1000, 6)  # 최종 클래스 수 (6)

    def forward(self, x):
        # 사전 학습된 모델에서 logits 추출
        outputs = self.base_model(x)
        x = outputs.logits  # logits만 가져옴

        # 사용자 정의 레이어를 통과
        x = self.fc1(x)
        x = nn.ReLU()(x)  # 활성화 함수 추가
        x = self.fc2(x)
        return x  # 최종 로짓 값 반환


In [34]:
#criterion에 적용할 데이터 분포 기반 가중치 조절

# 클래스별 샘플 수
class_counts = torch.tensor([175, 443, 163, 69, 6, 2], dtype=torch.float)

# 전체 샘플 수
total_samples = class_counts.sum()

# 클래스 가중치 계산
num_classes = len(class_counts)
class_weights = total_samples / (num_classes * class_counts)

print("Class Weights:", class_weights)

Class Weights: tensor([ 0.8171,  0.3228,  0.8773,  2.0725, 23.8333, 71.5000])


In [35]:
model = SkinNet().to(device)
#print(model)

In [36]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
def train(model, train_loader, optimizer):
    loss_total = 0
    correct_total = 0
    model.train()
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).long()

        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        loss.backward()
        optimizer.step()

        # 누적 손실 및 정확도 계산
        loss_total += loss.item()
        predicted_values = output.max(1, keepdim=True)[1]
        correct = predicted_values.eq(labels.view_as(predicted_values)).sum().item()
        correct_total += correct



        # 에폭 종료 후 평균 손실 및 정확도 계산
    loss_total /= len(train_loader)  # 배치 개수로 나누어 평균 계산
    accuracy = correct_total / len(train_loader.dataset)  # 전체 데이터셋에서 정확도 계산

    # 마지막 배치 후 결과 출력
    print(f"Train Finished - Train_Loss: {loss_total:.4f}, Train_Accuracy: {accuracy:.4f}")

    return loss_total, accuracy

In [26]:
def evaluate(model, test_loader):
    loss_total = 0
    correct_total = 0
    predict_history = []
    label_history = []
    total_count = 0
    batch_size = test_loader.batch_size  # DataLoader의 배치 크기 확인
    model.eval()  # evaluation mode로 설정 -> batch-normalization, drop-out 수행 중지
    with torch.no_grad():  # 가중치 업데이트 수행 중지
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device).long()
            output = model(images)
            loss = criterion(output, labels).item()
            loss_total += loss
            predicted_values = output.max(1, keepdim=True)[1]
            correct = predicted_values.eq(labels.view_as(predicted_values)).sum().item()
            correct_total += correct
            total_count += len(predicted_values)

            predict_history.append(predicted_values)
            label_history.append(labels)

    loss_total /= (total_count / batch_size)  # 배치 크기를 변수로 대체
    accuracy = correct_total / total_count

    return loss_total, accuracy, predict_history, label_history


In [37]:
# 학습 실행
for epoch in range(100):
    train(model, train_loader, optimizer)
    val_loss, val_accuracy, val_predict_history, val_label_history = evaluate(model, test_loader)
    print(f"Epoch : {epoch + 1}, val_Loss: {val_loss}, val_Accuracy : {val_accuracy}", '\n')

Train Finished - Train_Loss: 1.8494, Train_Accuracy: 0.2821
Epoch : 1, val_Loss: 2.4311984409795744, val_Accuracy : 0.2897196261682243 

Train Finished - Train_Loss: 1.6431, Train_Accuracy: 0.3660
Epoch : 2, val_Loss: 1.856267982554213, val_Accuracy : 0.19626168224299065 

Train Finished - Train_Loss: 1.5589, Train_Accuracy: 0.3112
Epoch : 3, val_Loss: 1.9304872316734814, val_Accuracy : 0.3644859813084112 

Train Finished - Train_Loss: 1.4961, Train_Accuracy: 0.3590
Epoch : 4, val_Loss: 1.5943128995806257, val_Accuracy : 0.2336448598130841 

Train Finished - Train_Loss: 1.4588, Train_Accuracy: 0.2890
Epoch : 5, val_Loss: 1.5631011713330991, val_Accuracy : 0.24299065420560748 

Train Finished - Train_Loss: 1.4698, Train_Accuracy: 0.3182
Epoch : 6, val_Loss: 1.7435548550614686, val_Accuracy : 0.17757009345794392 

Train Finished - Train_Loss: 1.4377, Train_Accuracy: 0.3380
Epoch : 7, val_Loss: 1.7507685367192063, val_Accuracy : 0.2616822429906542 

Train Finished - Train_Loss: 1.4343, Tr

In [38]:
# 학습 실행2
for epoch in range(100):
    train(model, train_loader, optimizer)
    val_loss, val_accuracy, val_predict_history, val_label_history = evaluate(model, test_loader)
    print(f"Epoch : {epoch + 1}, val_Loss: {val_loss}, val_Accuracy : {val_accuracy}", '\n')

Train Finished - Train_Loss: 1.0326, Train_Accuracy: 0.4732
Epoch : 1, val_Loss: 2.70676785763179, val_Accuracy : 0.2616822429906542 

Train Finished - Train_Loss: 1.0590, Train_Accuracy: 0.4814
Epoch : 2, val_Loss: 3.0651151710581557, val_Accuracy : 0.2897196261682243 

Train Finished - Train_Loss: 1.0387, Train_Accuracy: 0.4580
Epoch : 3, val_Loss: 2.530284988545926, val_Accuracy : 0.37383177570093457 

Train Finished - Train_Loss: 1.0202, Train_Accuracy: 0.4697
Epoch : 4, val_Loss: 2.96349256283769, val_Accuracy : 0.2897196261682243 

Train Finished - Train_Loss: 1.0414, Train_Accuracy: 0.4639
Epoch : 5, val_Loss: 3.2696960306613243, val_Accuracy : 0.35514018691588783 

Train Finished - Train_Loss: 1.0339, Train_Accuracy: 0.4674
Epoch : 6, val_Loss: 3.406126325375566, val_Accuracy : 0.308411214953271 

Train Finished - Train_Loss: 1.0317, Train_Accuracy: 0.4207
Epoch : 7, val_Loss: 2.9243244278096707, val_Accuracy : 0.32710280373831774 

Train Finished - Train_Loss: 1.0286, Train_Ac